In [1]:
pwd

'C:\\Users\\JiHee\\Desktop\\python\\크롤링'

In [2]:
import pandas as pd
import numpy as np

from selenium import webdriver  # 라이브러리(모듈) 가져오라
from selenium.webdriver import ActionChains as AC # 웹브라우저 동작
from tqdm import tqdm
from tqdm.notebook import tqdm
import re
from time import sleep
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

In [3]:
# 데이터 수집할 키워드 지정
keyword = "플레이리스트"

In [4]:
# 크롬창 띄우기
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(chrome_options=options)

#driver = webdriver.Chrome("chromedriver.exe")  # 맥 : /Users/aiden/Desktop/chromedriver
driver.get("https://www.youtube.com/results?search_query={}".format(keyword))
time.sleep(3)

<ipython-input-4-64ec43ed8d39>:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


In [5]:
# page1에서 url 수집하기
things = driver.find_elements_by_id('thumbnail')


url_list = []
for thing in things:
    url = thing.get_attribute('href')
    url_list.append(url)

In [6]:
df = pd.DataFrame({"url":url_list})
df.to_csv('YoutubeMusic_urls.csv')

In [7]:
# 저장해둔 url 불러오기
df = pd.read_csv('YoutubeMusic_urls.csv')
df['url']


0           https://www.youtube.com/watch?v=Z0OAEMK8NWA
1           https://www.youtube.com/watch?v=g7c0W-8rNmw
2           https://www.youtube.com/watch?v=BbT99O68AL0
3           https://www.youtube.com/watch?v=6wJepYuiznU
4           https://www.youtube.com/watch?v=lD5MHkAB_eQ
5           https://www.youtube.com/watch?v=T-E7OKQrfdU
6           https://www.youtube.com/watch?v=Tu8U2w7fLlU
7           https://www.youtube.com/watch?v=Q1v711XwKG4
8           https://www.youtube.com/watch?v=EVLfmnCmrGU
9           https://www.youtube.com/watch?v=FFnlKUbwDFs
10          https://www.youtube.com/watch?v=tH3Bv_w3I-g
11          https://www.youtube.com/watch?v=RYoOL4xXgR0
12          https://www.youtube.com/watch?v=UGtltoMYKNM
13          https://www.youtube.com/watch?v=wOpvxU5STwQ
14          https://www.youtube.com/watch?v=mxJGDa7ThbE
15          https://www.youtube.com/watch?v=JUeGb1JWW9E
16          https://www.youtube.com/watch?v=z5F48yvJVVE
17          https://www.youtube.com/watch?v=ppHV

In [12]:
dict = {}
# defined class ro identifted
title_class = 'title.style-scope.ytd-video-primary-info-renderer.style-scope.ytd-video-primary-info-renderer'

# 페이지당 동영상 수집
for i in tqdm(range(0, 5)):   # len(df['url'])
    try:
        options = webdriver.ChromeOptions()
        options.add_argument("--start-maximized")
        driver = webdriver.Chrome(chrome_options=options)
        driver.get(df['url'][i])

        last_height = driver.execute_script("return document.body.scrollHeight")

        # Element가 로딩될 때 까지 충분히 로딩을 가진다.
        wait = WebDriverWait(driver, 10)
        title = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, title_class))).text
        driver.execute_script("window.scrollTo(0, window.scrollY + 200);")
        channel = driver.find_element_by_xpath('//*[@id="meta"]').text.split('\n')[0]
        view = driver.find_element_by_xpath('//*[@id="count"]/ytd-video-view-count-renderer/span[1]').text[4:-1]
        like = driver.find_element_by_xpath('//*[@id="menu"]/ytd-menu-renderer[1]').text.split('\n')[0]
        bad = driver.find_element_by_xpath('//*[@id="menu"]/ytd-menu-renderer[1]').text.split('\n')[1]
        driver.execute_script("window.scrollTo(0, window.scrollY + 500);")

        time.sleep(3)
        review_count = driver.find_element_by_xpath('//*[@id="count"]/yt-formatted-string[1]/span[2]').text
        driver.execute_script("window.scrollTo(0, window.scrollY + 1000);")

        # youtube 전용 스크롤
        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        count = 0
        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(2.0)
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
            count = count + 1
            if new_page_height == last_page_height:
                break
            if count >= 2:
                break
            last_page_height = new_page_height

        review_list = []
        reviews = driver.find_elements_by_xpath('//*[@id="content"]/yt-formatted-string[2]')
        for review in reviews:    
            review = review.text
            review_list.append(review)

        target_info = {}
        target_info['동영상제목'] = title
        target_info['조회수'] = view
        target_info['채널'] = channel
        target_info['좋아요'] = like
        target_info['싫어요'] = bad
        target_info['댓글 수'] = review_count
        target_info['댓글'] = review_list

        dict[i] = target_info
        print(i, title)
        driver.close()
    except:
        print("error raised")
        driver.close()

<ipython-input-12-4b427912445b>:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


0 [ᴘʟᴀʏʟɪsᴛ] 노래뭐야? 너무좋잖아 띵곡 팝송 플레이리스트 Popsong Radio
1 [Playlist] 아니, 이런 노래는 어떻게 아는거야?? 너무 좋자나... | 노래 좀 듣는 애 플레이리스트😎ㅣShe has good taste in music😎
2 [𝐏𝐥𝐚𝐲𝐥𝐢𝐬𝐭] 이번 여름에 무조건 들어야하는 청량감 넘치는 아이돌 노래모음
3 [Playlist] 마음이 복잡하고 힘들 때 듣는 감성팝송 플레이리스트
4 [playlist] 다들 봄을 즐긴다는데, 내겐 쓰기만 해 l 노래모음 광고없음



In [9]:
# 판다스로 만들기
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df
# result_df.rename(columns={result_df.columns[0] : name}, inplace=True)

,동영상제목,조회수,채널,좋아요,싫어요,댓글 수,댓글
0,[ᴘʟᴀʏʟɪsᴛ] 노래뭐야? 너무좋잖아 띵곡 팝송 플레이리스트 Popsong Radio,"38,139",샵일삼Playlist#111,340,7,22,[# lhwook32@gmail.com\n\n0:01 ━───❍──────── 4...
1,"[Playlist] 아니, 이런 노래는 어떻게 아는거야?? 너무 좋자나... | 노...","3,957,668",essential;,7.6만,786,965,[앨범소개\n\n내 차 타는 친구들에게 들려주고 싶은 감각적인 플레이리스트. '노래...
2,[𝐏𝐥𝐚𝐲𝐥𝐢𝐬𝐭] 이번 여름에 무조건 들어야하는 청량감 넘치는 아이돌 노래모음,"123,024",Mean Playlist_민뮤직,1.3천,25,59,[영상이 마음에 들었으면 구독[ 💜 ] 과 좋아요[ 👍 ] 부탁드립니다~! 😊\n\...
3,[Playlist] 마음이 복잡하고 힘들 때 듣는 감성팝송 플레이리스트,"8,443,431",feel it,12만,1.3천,"12,085","[[Playlist]\n\n{35:41}, 화내면""인성 나오네""\n울면""약한척하네""..."
4,"[playlist] 다들 봄을 즐긴다는데, 내겐 쓰기만 해 l 노래모음 광고없음","33,320",눈에띠네,좋아요,싫어요,70,[예약 건다는걸 잘못 올려버렸네 \n사랑한다. 이번주도 화이팅 :) \n\nTIME...


In [10]:
# 파일로 저장
result_df.to_excel("YoutubeMusic_({}).xlsx".format(keyword))